In [0]:
sqlContext
from pyspark.sql.functions import *

In [0]:
orders_1 = spark.read.csv('/FileStore/tables/indian-takeaway-orders/restaurant_1_orders.csv', header=True)

orders_1 = orders_1.withColumn('Quantity', orders_1['Quantity'].cast('integer')).withColumn('Product Price', orders_1['Product Price'].cast('float')).withColumn('Total products', orders_1['Total products'].cast('integer')).withColumn('Order Date', to_timestamp('Order Date', 'dd/MM/yyyy HH:mm'))

orders_2 = spark.read.csv('/FileStore/tables/indian-takeaway-orders/restaurant_2_orders.csv', header=True)

orders_2 = orders_2.withColumn('Quantity', orders_2['Quantity'].cast('integer')).withColumn('Product Price', orders_2['Product Price'].cast('float')).withColumn('Total products', orders_2['Total products'].cast('integer')).withColumn('Order Date', to_timestamp('Order Date', 'dd/MM/yyyy HH:mm'))

orders_1.show()
orders_2.show()

+------------+-------------------+--------------------+--------+-------------+--------------+
|Order Number|         Order Date|           Item Name|Quantity|Product Price|Total products|
+------------+-------------------+--------------------+--------+-------------+--------------+
|       16118|2019-08-03 20:25:00|       Plain Papadum|       2|          0.8|             6|
|       16118|2019-08-03 20:25:00|    King Prawn Balti|       1|        12.95|             6|
|       16118|2019-08-03 20:25:00|         Garlic Naan|       1|         2.95|             6|
|       16118|2019-08-03 20:25:00|       Mushroom Rice|       1|         3.95|             6|
|       16118|2019-08-03 20:25:00| Paneer Tikka Masala|       1|         8.95|             6|
|       16118|2019-08-03 20:25:00|       Mango Chutney|       1|          0.5|             6|
|       16117|2019-08-03 20:17:00|          Plain Naan|       1|          2.6|             7|
|       16117|2019-08-03 20:17:00|       Mushroom Rice|     

In [0]:
prices_1 = spark.read.csv('/FileStore/tables/indian-takeaway-orders/restaurant_1_products_price.csv', header=True)
prices_1 = prices_1.withColumn('Product Price', prices_1['Product Price'].cast('float'))
prices_2 = spark.read.csv('/FileStore/tables/indian_takeaways/indian-takeaway-orders/restaurant_2_products_price.csv', header=True)
prices_2 = prices_2.withColumn('Product Price', prices_2['Product Price'].cast('float'))

prices_1.show()
prices_2.show()

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|   Mint Sauce|          0.5|
|  Lime Pickle|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|Onion Chutney|          0.5|
|Plain Papadum|          0.8|
|Spicy Papadum|          0.8|
|      Chapati|         1.95|
|        Puree|         1.95|
|Tandoori Roti|          2.6|
|   Plain Naan|          2.6|
|       Raitha|         2.95|
|   Keema Naan|         2.95|
|Peshwari Naan|         2.95|
|   Pilau Rice|         2.95|
|  Green Salad|         2.95|
|  Bottle Coke|         2.95|
|   Plain Rice|         2.95|
|  Garlic Naan|         2.95|
|   Onion Naan|         2.95|
+-------------+-------------+
only showing top 20 rows

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|Onion Chutney|          0.5|
|   Mint Sauce|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|  Lime Pickle|          0.5|
|Plain Papadum

In [0]:
orders_1.createOrReplaceTempView('orders_1')
prices_1.createOrReplaceTempView('prices_1')
orders_2.createOrReplaceTempView('orders_2')
prices_2.createOrReplaceTempView('prices_2')

In [0]:
# Unite orders and prices.

orders = orders_1.union(orders_2)
orders.show()

prices = prices_1.union(prices_2).distinct() # No duplicates
prices.show()

+------------+-------------------+--------------------+--------+-------------+--------------+
|Order Number|         Order Date|           Item Name|Quantity|Product Price|Total products|
+------------+-------------------+--------------------+--------+-------------+--------------+
|       16118|2019-08-03 20:25:00|       Plain Papadum|       2|          0.8|             6|
|       16118|2019-08-03 20:25:00|    King Prawn Balti|       1|        12.95|             6|
|       16118|2019-08-03 20:25:00|         Garlic Naan|       1|         2.95|             6|
|       16118|2019-08-03 20:25:00|       Mushroom Rice|       1|         3.95|             6|
|       16118|2019-08-03 20:25:00| Paneer Tikka Masala|       1|         8.95|             6|
|       16118|2019-08-03 20:25:00|       Mango Chutney|       1|          0.5|             6|
|       16117|2019-08-03 20:17:00|          Plain Naan|       1|          2.6|             7|
|       16117|2019-08-03 20:17:00|       Mushroom Rice|     

In [0]:
# What is the most popular item ordered
# Find sum quantity of products ordered within an order.

groupby = orders.groupBy('Item Name').sum('Quantity')
orderby = groupby.orderBy(groupby['sum(Quantity)'].desc())
orderby.show()

+--------------------+-------------+
|           Item Name|sum(Quantity)|
+--------------------+-------------+
|       Plain Papadum|        28704|
|          Pilau Rice|        18121|
|                Naan|         8730|
|         Garlic Naan|         8127|
|       Mango Chutney|         6628|
|          Plain Rice|         6496|
|Chicken Tikka Masala|         6443|
|         Bombay Aloo|         6167|
|          Mint Sauce|         5048|
|       Mushroom Rice|         5039|
|          Plain Naan|         4983|
|          Keema Naan|         4358|
|             Chapati|         4326|
|       Peshwari Naan|         4195|
|         Onion Bhaji|         3985|
|       Onion Chutney|         3595|
|               Korma|         3361|
|           Saag Aloo|         3323|
|      Butter Chicken|         2761|
|              Madras|         2758|
+--------------------+-------------+
only showing top 20 rows



In [0]:
# Most popular item is a Plain Papadum
# When is this item usually ordered?

where = orders.select('*').where(col('Item Name') == 'Plain Papadum')
dates = where.groupBy('Order Date').count()
dates_asc = dates.orderBy(dates['Order Date'].asc())
dates.show()

+-------------------+-----+
|         Order Date|count|
+-------------------+-----+
|2019-08-03 19:26:00|    1|
|2019-06-22 19:27:00|    2|
|2019-06-01 19:42:00|    1|
|2019-03-29 19:19:00|    1|
|2018-10-22 19:20:00|    1|
|2019-03-01 20:23:00|    1|
|2018-12-01 19:37:00|    1|
|2018-12-01 19:07:00|    1|
|2018-02-25 18:44:00|    2|
|2018-01-05 17:43:00|    1|
|2017-10-21 18:19:00|    1|
|2017-09-09 13:45:00|    1|
|2017-07-21 19:24:00|    1|
|2017-06-25 17:59:00|    2|
|2017-04-21 20:11:00|    1|
|2016-03-20 17:40:00|    1|
|2019-07-14 20:19:00|    1|
|2019-06-13 21:12:00|    1|
|2019-02-01 18:23:00|    1|
|2019-01-12 19:04:00|    1|
+-------------------+-----+
only showing top 20 rows



In [0]:
# Extract hour

hours = dates.withColumn('hour', hour(col('Order Date')))

# Group by hour

hourly_avg = hours.groupBy('hour').sum('count').orderBy('hour')
hourly_avg.show()

+----+----------+
|hour|sum(count)|
+----+----------+
|   1|         3|
|   2|         1|
|   8|         1|
|   9|         2|
|  10|         7|
|  11|        16|
|  12|        71|
|  13|        63|
|  14|        34|
|  15|        61|
|  16|       299|
|  17|      2007|
|  18|      3189|
|  19|      2612|
|  20|      1085|
|  21|       436|
|  22|       139|
|  23|         4|
+----+----------+



In [0]:
# Display maximum

maximum = hourly_avg.select(max('sum(count)'))
max_hour = hourly_avg.join(maximum, hourly_avg['sum(count)'] == maximum['max(sum(count))'], 'inner').select('hour', 'sum(count)')
max_hour.show()

+----+----------+
|hour|sum(count)|
+----+----------+
|  18|      3189|
+----+----------+



In [0]:
# Highest number of Papadum orders just after 18:00
# How much money was made in this hour?

unit_price = prices.select('Item Name', 'Product Price').where(col('Item Name') == 'Plain Papadum')
mh_with_name = max_hour.withColumn('Item Name', lit('Plain Papadum'))
all_info = unit_price.join(mh_with_name, unit_price['Item Name'] == mh_with_name['Item Name'], 'inner')
all_info = all_info.withColumn('Total', col('sum(count)') * col('Product Price'))
all_info.select('Total').show()

+------+
| Total|
+------+
|2551.2|
+------+



In [0]:
# Total price of orders of Papadums at 18:00 is £2551.20